In [1]:
import numpy as np
from pandas import isna
from scipy.sparse import issparse, linalg, spdiags
import anndata

import scvelo as scv
import scanpy as sc
from velocity_graph import velocity_graph
import lvpt

from scvelo import logging as logg
from scvelo.preprocessing.moments import get_connectivities
from scvelo.tools.terminal_states import terminal_states
from scvelo.utils import groups_to_bool, scale, strings_to_categoricals

In [2]:
adata = anndata.read_h5ad("D:/L/data/example_data/simulated/linear.h5ad")

if 'counts_spliced' in adata.layers:
    adata.layers['spliced'] = adata.layers.pop('counts_spliced')
if 'counts_unspliced' in adata.layers:
    adata.layers['unspliced'] = adata.layers.pop('counts_unspliced')
    
sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)
sc.tl.pca(adata, svd_solver='arpack')

sc.pp.neighbors(adata)
sc.tl.umap(adata)

In [ ]:
adata = anndata.read_h5ad("D:/L/data/example_data/simulated/Dyngen/Bifurcating.h5ad")
sc.pp.neighbors(adata, n_neighbors = 13,n_pcs = 20)

In [ ]:
adata = anndata.read_h5ad("D:/L/data/example_data/simulated/Dyngen/Trifurcating.h5ad")
sc.pp.neighbors(adata, n_neighbors = 13,n_pcs = 20)

In [ ]:
adata = anndata.read_h5ad("D:/L/data/example_data/simulated/trifurcating.h5ad")

if 'counts_spliced' in adata.layers: 
    adata.layers['spliced'] = adata.layers.pop('counts_spliced') 
if 'counts_unspliced' in adata.layers: 
    adata.layers['unspliced'] = adata.layers.pop('counts_unspliced')

sc.tl.pca(adata, svd_solver='arpack')
sc.pp.neighbors(adata, n_neighbors =30,n_pcs = 20)
sc.tl.umap(adata) 

In [2]:
adata = anndata.read_h5ad("C:/Users/user/Desktop/TRY/CellPath-master/example_data/simulated/cycletree.h5ad")

In [3]:
scv.tl.velocity(adata)
velocity_graph(adata, steady=0.06, approx=True)
lvpt.velocity_pseudotime(adata)

Normalized count data: spliced, unspliced.
computing neighbors
    finished (0:00:00) --> added 
    'distances' and 'connectivities', weighted adjacency matrices (adata.obsp)
computing moments based on connectivities
    finished (0:00:00) --> added 
    'Ms' and 'Mu', moments of un/spliced abundances (adata.layers)
computing velocities
    finished (0:00:00) --> added 
    'velocity', velocity vectors for each individual cell (adata.layers)
computing velocity graph (using 1/4 cores)


  0%|          | 0/1000 [00:00<?, ?cells/s]

    finished (0:00:02) --> added 
    'velocity_graph', sparse matrix with cosine correlations (adata.uns)
computing terminal states
    identified 2 regions of root cells and 3 regions of end points .
    finished (0:00:00) --> added
    'root_cells', root cells of Markov diffusion process (adata.obs)
    'end_points', end points of Markov diffusion process (adata.obs)


In [4]:
def kendalltau(pt_pred, pt_true):
    """\
    Description
        kendall tau correlationship
    
    Parameters
    ----------
    pt_pred
        inferred pseudo-time
    pt_true
        ground truth pseudo-time
    Returns
    -------
    tau
        returned score
    """
    from scipy.stats import kendalltau
    pt_true = pt_true.squeeze()
    pt_pred = pt_pred.squeeze()
    tau, p_val = kendalltau(pt_pred, pt_true)
    return tau

from scipy.stats import spearmanr
import numpy as np

def spearmanr_corr(pt_pred, pt_true):
    """\
    Description
        Spearman's rank correlation coefficient
    
    Parameters
    ----------
    pt_pred
        inferred pseudo-time
    pt_true
        ground truth pseudo-time
    Returns
    -------
    rho
        Spearman correlation coefficient
    """
    pt_true = pt_true.squeeze()
    pt_pred = pt_pred.squeeze()
    rho, p_val = spearmanr(pt_pred, pt_true)
    return rho

C:\jupter\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [8]:
kt = kendalltau(adata._obs.velocity_pseudotime,adata._obs.sim_time)
kt

0.6587224692439201

In [9]:
sm = spearmanr_corr(adata._obs.velocity_pseudotime,adata._obs.sim_time)
sm

0.6296857322271471